In [15]:
# import numpy library and assign it to a parameter
import numpy as np
import parameters, grid, matrices

params = parameters.Params()
grid = grid.Grid(params)
mat = matrices.Matrices(params, grid)

# connecting the parameters under total current calculation to numpy functions
b = params.b
delta = params.delta
DI = params.DI
epsp = params.epsp
G0 = params.G0
Kn = params.Kn
Kp = params.Kp
lam = params.lam
lam2 = params.lam2
N = params.N
NE = params.NE
NH = params.NH
N0 = params.N0
q = params.q
sigma = params.sigma
Tion = params.Tion
Vt = params.Vt

dx = grid.dx

Av = mat.Av
Dx = mat.Dx

#define new function total current simulated
def total_current_slm(t_vector, u_matrix, *model):
    
    P = u_matrix[0:N+1,:]
    phi = u_matrix[N+1:2*N+2,:]
    n = u_matrix[2*N+2:3*N+3,:]
    p = u_matrix[3*N+3:4*N+4,:]
    
    mE_data = np.zeros((N, t_vector.size))
    dd_data = np.zeros((N, t_vector.size))
    FP_data = np.zeros((N, t_vector.size))
    fn_data = np.zeros((N, t_vector.size))
    fp_data = np.zeros((N, t_vector.size))
    dt = np.append([0.0], np.diff(t_vector))
    
    for i in range(0, t_vector.size):
        mE = Dx@phi[:,i]
        dd = mE*dt[i]
        FP = lam*(Dx@P[:,i] + mE*(Av@P[:,i]))
        fn = (Dx@n[:,i] - mE*(Av@n[:,i]))
        fp = -(Dx@p[:,i] + mE*(Av@p[:,i]))
        mE_data[:,i] = mE
        dd_data[:,i] = dd
        FP_data[:,i] = FP
        fn_data[:,i] = fn
        fp_data[:,i] = fp
    
    J_data = fn_data + fp_data - (sigma*lam2/delta)*dd_data + (sigma*lam/delta)*FP_data
    midpoint = int(np.ceil((N+1)/2))
    J_total = J_data[midpoint,:]

    return J_total, J_data, FP_data, dd_data, fn_data, fp_data

def total_current_tlm(t_vector, u_matrix, *model):
    P = u_matrix[0:N+1,:]
    phi = u_matrix[N+1:2*N+2,:]
    n = u_matrix[2*N+2:3*N+3,:]
    p = u_matrix[3*N+3:4*N+4,:]
    mE_data = np.zeros((N, t_vector.size))
    dd_data = np.zeros((N, t_vector.size))
    FP_data = np.zeros((N, t_vector.size))
    fn_data = np.zeros((N, t_vector.size))
    fp_data = np.zeros((N, t_vector.size))
    dt = np.append([0.0], np.diff(t_vector))

    for i in range(0, t_vector.size):
        mE = Dx@phi[:,i]
        dd = mE*dt[i]
        FP = lam*(Dx@P[:,i] + mE*(Av@P[:,i]))
        fn = Kn*(Dx@n[:,i] - mE*(Av@n[:,i]))
        fp = -Kp*(Dx@p[:,i] + mE*(Av@p[:,i]))
    
        mE_data[:,i] = mE
        dd_data[:,i] = dd
        FP_data[:,i] = FP
        fn_data[:,i] = fn
        fp_data[:,i] = fp
        
    J_data = fn_data + fp_data - (epsp*Vt/(q*G0*b**2*Tion))*dd_data + (DI*N0/(G0*b**2))*FP_data
    midpoint = int(np.ceil((N+1)/2))
    J_total = J_data[midpoint,:]
    return J_total, J_data, FP_data, dd_data, fn_data, fp_data

ModuleNotFoundError: No module named 'parameters'